In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import iplot
import plotly.figure_factory as ff

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score,confusion_matrix


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'plotly'

In [ ]:
## Set random seed for reproducibility
def set_seed(seed=423):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed()

**Data Ingestion**

In [ ]:
# 2.1 Define file paths
train_path = '/kaggle/input/reviews-2025/amazon_review_ID.shuf.lrn.csv'
test_path  = '/kaggle/input/reviews-2025/amazon_review_ID.shuf.tes.csv'
val_path = '/kaggle/input/reviews-2025/amazon_review_ID.shuf.sol.ex.csv'

In [ ]:
# 2.2 Load with pandas
train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)
final_test_df = pd.read_csv(test_path)
display(train_df.head())
print(train_df.info())
print("Test set shape:", test_df.shape)

In [ ]:
classes = train_df["Class"].unique()
print(len(classes))

**Exploratory Data Analysis (EDA)**

In [ ]:
import pandas as pd

class EDAOverview:
    """
    Prints total counts of 'y', 'n', '?', and missing values
    across all feature columns (drops target column if present).
    """
    def __init__(self, train_df: pd.DataFrame, target_col: str = 'Class'):
        # Drop the target column if present
        self.df = train_df.drop(columns=[target_col], errors='ignore')
        self._print_summary()

    def _print_summary(self):
        flat = self.df.values.flatten()
        total_cells = flat.size
        count_y = (flat == 'y').sum()
        count_n = (flat == 'n').sum()
        count_q = (flat == '?').sum()
        count_missing = pd.isnull(flat).sum()

        print("=== EDA Summary ===")
        print(f"Total cells       : {total_cells:,}")
        print(f"Count 'y'         : {count_y:,} ({count_y/total_cells:.1%})")
        print(f"Count 'n'         : {count_n:,} ({count_n/total_cells:.1%})")
        print(f"Count '?'         : {count_q:,} ({count_q/total_cells:.1%})")
        print(f"Missing values    : {count_missing:,} ({count_missing/total_cells:.1%})")
        print("====================")


overview = EDAOverview(train_df, target_col='Class')


In [ ]:
x = train_df['Class'].value_counts()
labels = train_df['Class'].unique().tolist()
print(len(labels))
className = go.Pie(labels=labels,values=x,hoverinfo='label+percent',textinfo='value')
iplot([className]) 

**DATA SPLIT**

In [ ]:
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in split.split(train_df,train_df['Class']):
    strat_train_set = train_df.loc[train_index]
    strat_test_set = train_df.loc[test_index]
strat_test_set['Class'].value_counts() / len(strat_test_set) 

**TRAINING**

In [ ]:
reviews = strat_train_set.drop('Class',axis=1)
reviews_labels = strat_train_set['Class'].copy()

reviews_test = strat_test_set.drop('Class',axis=1)
reviews_test_labels = strat_test_set['Class'].copy()

In [ ]:
ordinal_encoder = OrdinalEncoder()
reviews_cat_encoded = ordinal_encoder.fit_transform(reviews)
reviews_cat_encoded

ordinal_encoder = OrdinalEncoder()
reviews_test_cat_encoded = ordinal_encoder.fit_transform(reviews_test)
reviews_test_cat_encoded

reviews_validation_encoded = ordinal_encoder.fit_transform(test_df)
reviews_validation_encoded 

In [ ]:
# define param-grids
param_grids = {
    'KNN': {
        'n_neighbors': [3,5,7,9],
        'weights': ['uniform','distance'],
        'p': [1,2]  # Manhattan vs Euclidean
    },
    'DecisionTree': {
        'max_depth': [None, 5,10,20],
        'min_samples_split': [2,5,10],
        'criterion': ['gini','entropy']
    },
    'MLP': {
        'hidden_layer_sizes': [(50,),(100,),(50,50)],
        'activation': ['relu','tanh','logistic'],
        'alpha': [1e-4, 1e-3, 1e-2]
    }
}

In [ ]:
## wrap each in GridSearchCV
searches = {}

searches['KNN'] = GridSearchCV(
    KNeighborsClassifier(),
    param_grids['KNN'],
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

searches['DecisionTree'] = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grids['DecisionTree'],
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

searches['MLP'] = GridSearchCV(
    MLPClassifier(max_iter=500, random_state=42),
    param_grids['MLP'],
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)


**AMBITIOUS, FIT EACH**

In [ ]:
# 3) fit each
best_estimators = {}
for name, gs in searches.items():
    print(f"\n➡️ Running grid-search for {name} …")
    gs.fit(reviews_cat_encoded, reviews_labels)
    best_estimators[name] = gs.best_estimator_
    print(f"▶️ Best {name} params: {gs.best_params_}")
    print(f"▶️ CV accuracy: {gs.best_score_:.3f}")

In [ ]:
# --- 1) KNN ---
knn_best = KNeighborsClassifier(
    n_neighbors=7,
    p=1,
    weights='distance'
)

knn_best.fit(reviews, reviews_labels)
knn_pred = knn_best.predict(reviews_test)
print("KNN Test Accuracy: {:.3f}".format(accuracy_score(reviews_test_labels, knn_pred)))
print(classification_report(reviews_test_labels, knn_pred, zero_division=0))


In [ ]:
# --- 2) Decision Tree ---
dt_best = DecisionTreeClassifier(
    criterion='gini',
    max_depth=None,
    min_samples_split=10,
    random_state=42
)
dt_best.fit(reviews, reviews_labels)
dt_pred = dt_best.predict(reviews_test)
print("Decision Tree Test Accuracy: {:.3f}".format(accuracy_score(reviews_test_labels, dt_pred)))
print(classification_report(reviews_test_labels, dt_pred, zero_division=0))


In [ ]:
# --- 3) MLP ---
mlp_best = MLPClassifier(
    activation='logistic',
    alpha=0.0001,
    hidden_layer_sizes=(100,),
    max_iter=1000,
    random_state=42
)
mlp_best.fit(reviews, reviews_labels)
mlp_pred = mlp_best.predict(reviews_test)
print("MLP Test Accuracy: {:.3f}".format(accuracy_score(reviews_test_labels, mlp_pred)))
print(classification_report(reviews_test_labels, mlp_pred, zero_division=0))

In [ ]:
final_preds = mlp_best.predict(reviews_test)
submission = pd.DataFrame({
    'ID': reviews_test['ID'],
    'class': final_preds
})
submission.to_csv('solution_file.csv', index=False)